## get file basic information

one big tar file -> multiple gz files -> multiple txt files -> then what?
- spark에서 여러 파일을 하나의 데이터프레임으로 읽는다
- hdfs가 대용량의 하나의 파일 분산 write하는거에 적절하긴한데.. 그럼 hdfs에 하나의 파일로 적재해놓고 읽는 과정이 한번 더 수반죄는거니까 굳이?
- 근데 또 spark에서 자주 읽는다고 하면 여러개로 쪼개져있는데 iterable하게 읽는것보다 hdfs에 최초 한번만 단일 파일로 만들어놓고 이거 spark에서 읽는게 맞는것같기도하고...]

In [35]:
os.listdir('./data')

['000004_0', '000000_0', '000003_0', '000002_0', '000005_0', '000001_0']

In [37]:
import os

path = './data'
for file in os.listdir(path):
  try:
    n = os.path.getsize(os.path.join(path, file))
    print(file)
    print('file : ', n, "Bytes")                           # 바이트 단위로 구하기
    print('file : ', n / 1024, "KB")                       # 킬로바이트 단위로
    print('file : ', "%.2f MB" % (n / (1024.0 * 1024.0)))  # 메가바이트 단위로
    print('file : ', "%.2f GB" % (n / (1024.0 * 1024.0 * 1000.0)))  # 기가바이트 단위로
  except os.error:
    print ("파일이 없거나 에러입니다.")

000004_0
file :  940095952 Bytes
file :  918062.453125 KB
file :  896.55 MB
file :  0.90 GB
000000_0
file :  939487083 Bytes
file :  917467.8544921875 KB
file :  895.96 MB
file :  0.90 GB
000003_0
file :  938279749 Bytes
file :  916288.8173828125 KB
file :  894.81 MB
file :  0.89 GB
000002_0
file :  935025145 Bytes
file :  913110.4931640625 KB
file :  891.71 MB
file :  0.89 GB
000005_0
file :  935502127 Bytes
file :  913576.2958984375 KB
file :  892.16 MB
file :  0.89 GB
000001_0
file :  937993267 Bytes
file :  916009.0498046875 KB
file :  894.54 MB
file :  0.89 GB


In [ ]:
#일단 제발 읽기나 해 하나라도

In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = SparkSession.builder\
.appName("spark-local-environment-test")\
.config('spark.jars', '$SPARK_HOME/jars/hadoop-aws-3.2.0.jar')\
.config('spark.jars', '$SPARK_HOME/jars/aws-java-sdk-bundle-1.11.375.jar')\
.getOrCreate()

spark

In [5]:
!ls data

000000_0  000001_0  000002_0  000003_0	000004_0  000005_0


In [6]:
data_path = './data/000000_0'

`adid`tab{`pacakge_name`:`install_market`:`dt1`:`dt2`:`dt3`:`sec_d`:`sec_month`,}*tab`carrier`tab`device_model`tab`device_os`tab`gender`tab`age`

### tab 으로 구분된 애들 먼저 제일 상위 덩어리로 읽기

In [7]:
from pyspark.sql.types import StructType, StructField, StringType

schema_with_tab = StructType(
    [
        StructField("adid", StringType(), True),
        StructField("adid_details", StringType(), True),
        StructField("carrier", StringType(), True),
        StructField("device_model", StringType(), True),
        StructField("device_os", StringType(), True),
        StructField("gender", StringType(), True),
        StructField("age", StringType(), True),
    ]
)

df = (
    spark.read
        .option("inferSchema", "true")
        .option("delimiter", "\t")
        .option("multiLine", "true")
        .option("header", "false")
        .option("encoding", "utf-8")
        .csv(data_path, schema=schema_with_tab)
)

In [21]:
display(df)

adid,adid_details,carrier,device_model,device_os,gender,age
0000237c-84d5-42c...,com.google.androi...,SKT,SM-G986N,13.0,Z,ZZ
00002a34-4418-430...,com.sec.android.e...,SKT,SM-N981N,13.0,Z,ZZ
00018a31-b0f1-4ec...,com.kt.aljjapackp...,KT,SM-G991N,14.0,Z,40
0001df59-3d34-415...,com.creasmall:G:2...,SKT,SM-S906N,14.0,Z,ZZ
0002ca60-d565-442...,mok.android:G:202...,SKT,SM-S911N,14.0,Z,ZZ
00032f56-a035-476...,com.google.androi...,ZZZ,SM-G930S,8.0,Z,ZZ
00034f01-be1c-4da...,com.google.androi...,KT,SM-M446K,14.0,Z,ZZ
0003b234-16b6-40d...,com.android.chrom...,KT,SM-G960N,10.0,Z,ZZ
00040286-9aaf-417...,com.ahnlab.v3mobi...,ZZZ,LM-G900N,11.0,Z,ZZ
0004489b-a47d-455...,com.sktelecom.vco...,SKT,SM-S906N,14.0,Z,30


tab 구분자로 구분된 데이터 안에 nested - comma로 구분된 정보

In [10]:
df.limit(1).collect() 

[Row(adid='0000237c-84d5-42cf-9be4-80a4522de0c9', adid_details='com.google.android.youtube:G:2009010100:2024082204:2024083007:0:2084,teamdoppelganger.smarterbus:G:2021063018:2024052017:2024052017:0:0,com.google.android.googlequicksearchbox:G:2009010100:2024081804:1970010109:0:0,com.samsung.android.app.tips:S:2022020601:2023062301:1970010109:0:0,org.adblockplus.adblockplussbrowser:S:2021090619:2023120915:1970010109:0:0,com.nhn.android.nbooks:G:2022111223:2024042503:2024041020:0:0,kr.go.hrd.app:G:2020102008:2023042121:1970010109:0:0,com.kebhana.hanapush:G:2020122120:2024080401:2024082202:0:0,net.daum.android.daum:G:2020042322:2024080401:2024083123:6549:17329,com.seeon.uticket:G:2022041915:2024082204:2024083012:0:184,com.yagins.hiraportal:G:2023091210:2023091210:1970010109:0:0,net.orizinal.subway:G:2020043011:2024081312:2024082917:0:173,skplanet.musicmate:O:2020042214:2021111308:1970010109:0:0,com.microsoft.office.officehubrow:G:2009010100:2024020416:1970010109:0:0,com.sec.android.app.pop

### 하위 덩어리 분리
#### 리스트 타입으로 만들기
"install_market:dt1:dt2:dt3:sec_d:sec_month, install_market:dt1:dt2:dt3:sec_d:sec_month, install_market:dt1:dt2:dt3:sec_d:sec_month" 이런식으로 되어있는 string comma 기준으로 split해서 리스트 타입 splited 컬럼 생성

In [13]:
df = df.withColumn('splited', F.split(df["adid_details"], ","))
display(df.show())

+--------------------+--------------------+-------+------------+---------+------+---+--------------------+
|                adid|        adid_details|carrier|device_model|device_os|gender|age|             splited|
+--------------------+--------------------+-------+------------+---------+------+---+--------------------+
|0000237c-84d5-42c...|com.google.androi...|    SKT|    SM-G986N|     13.0|     Z| ZZ|[com.google.andro...|
|00002a34-4418-430...|com.sec.android.e...|    SKT|    SM-N981N|     13.0|     Z| ZZ|[com.sec.android....|
|00018a31-b0f1-4ec...|com.kt.aljjapackp...|     KT|    SM-G991N|     14.0|     Z| 40|[com.kt.aljjapack...|
|0001df59-3d34-415...|com.creasmall:G:2...|    SKT|    SM-S906N|     14.0|     Z| ZZ|[com.creasmall:G:...|
|0002ca60-d565-442...|mok.android:G:202...|    SKT|    SM-S911N|     14.0|     Z| ZZ|[mok.android:G:20...|
|00032f56-a035-476...|com.google.androi...|    ZZZ|    SM-G930S|      8.0|     Z| ZZ|[com.google.andro...|
|00034f01-be1c-4da...|com.google.andr

None

In [18]:
df.select(F.col("splited")).limit(1).collect()

[Row(splited=['com.google.android.youtube:G:2009010100:2024082204:2024083007:0:2084', 'teamdoppelganger.smarterbus:G:2021063018:2024052017:2024052017:0:0', 'com.google.android.googlequicksearchbox:G:2009010100:2024081804:1970010109:0:0', 'com.samsung.android.app.tips:S:2022020601:2023062301:1970010109:0:0', 'org.adblockplus.adblockplussbrowser:S:2021090619:2023120915:1970010109:0:0', 'com.nhn.android.nbooks:G:2022111223:2024042503:2024041020:0:0', 'kr.go.hrd.app:G:2020102008:2023042121:1970010109:0:0', 'com.kebhana.hanapush:G:2020122120:2024080401:2024082202:0:0', 'net.daum.android.daum:G:2020042322:2024080401:2024083123:6549:17329', 'com.seeon.uticket:G:2022041915:2024082204:2024083012:0:184', 'com.yagins.hiraportal:G:2023091210:2023091210:1970010109:0:0', 'net.orizinal.subway:G:2020043011:2024081312:2024082917:0:173', 'skplanet.musicmate:O:2020042214:2021111308:1970010109:0:0', 'com.microsoft.office.officehubrow:G:2009010100:2024020416:1970010109:0:0', 'com.sec.android.app.popupcalcu

#### adid 키로 주고 리스트 타입 컬럼 각 레코드로 explode

In [27]:
exploded_df = df.select(F.explode(df["splited"]).alias('ad_id_details'), df['adid'])
exploded_df.show()

+--------------------+--------------------+
|       ad_id_details|                adid|
+--------------------+--------------------+
|com.google.androi...|0000237c-84d5-42c...|
|teamdoppelganger....|0000237c-84d5-42c...|
|com.google.androi...|0000237c-84d5-42c...|
|com.samsung.andro...|0000237c-84d5-42c...|
|org.adblockplus.a...|0000237c-84d5-42c...|
|com.nhn.android.n...|0000237c-84d5-42c...|
|kr.go.hrd.app:G:2...|0000237c-84d5-42c...|
|com.kebhana.hanap...|0000237c-84d5-42c...|
|net.daum.android....|0000237c-84d5-42c...|
|com.seeon.uticket...|0000237c-84d5-42c...|
|com.yagins.hirapo...|0000237c-84d5-42c...|
|net.orizinal.subw...|0000237c-84d5-42c...|
|skplanet.musicmat...|0000237c-84d5-42c...|
|com.microsoft.off...|0000237c-84d5-42c...|
|com.sec.android.a...|0000237c-84d5-42c...|
|com.samsung.andro...|0000237c-84d5-42c...|
|com.lh.apply:G:20...|0000237c-84d5-42c...|
|com.samsung.andro...|0000237c-84d5-42c...|
|com.google.androi...|0000237c-84d5-42c...|
|com.sampleapp:G:2...|0000237c-8

In [28]:
exploded_df.limit(1).collect()

[Row(ad_id_details='com.google.android.youtube:G:2009010100:2024082204:2024083007:0:2084', adid='0000237c-84d5-42cf-9be4-80a4522de0c9')]

In [31]:
exploded_df = exploded_df.withColumn('splited', F.split(exploded_df["ad_id_details"], ":"))
exploded_df.limit(30).select('splited').collect()

[Row(splited=['com.google.android.youtube', 'G', '2009010100', '2024082204', '2024083007', '0', '2084']),
 Row(splited=['teamdoppelganger.smarterbus', 'G', '2021063018', '2024052017', '2024052017', '0', '0']),
 Row(splited=['com.google.android.googlequicksearchbox', 'G', '2009010100', '2024081804', '1970010109', '0', '0']),
 Row(splited=['com.samsung.android.app.tips', 'S', '2022020601', '2023062301', '1970010109', '0', '0']),
 Row(splited=['org.adblockplus.adblockplussbrowser', 'S', '2021090619', '2023120915', '1970010109', '0', '0']),
 Row(splited=['com.nhn.android.nbooks', 'G', '2022111223', '2024042503', '2024041020', '0', '0']),
 Row(splited=['kr.go.hrd.app', 'G', '2020102008', '2023042121', '1970010109', '0', '0']),
 Row(splited=['com.kebhana.hanapush', 'G', '2020122120', '2024080401', '2024082202', '0', '0']),
 Row(splited=['net.daum.android.daum', 'G', '2020042322', '2024080401', '2024083123', '6549', '17329']),
 Row(splited=['com.seeon.uticket', 'G', '2022041915', '2024082204'

In [38]:
exploded_df = (
    exploded_df.withColumn("package_name", F.col("splited").getItem(0))
    .withColumn("install_market", F.col("splited").getItem(1))
    .withColumn("first_install_dt", F.col("splited").getItem(2))
    .withColumn("last_update_dt", F.col("splited").getItem(3))
    .withColumn("last_use_dt", F.col("splited").getItem(4))
    .withColumn("daily_use_sec", F.col("splited").getItem(5))
    .withColumn("weekly_use_sec", F.col("splited").getItem(6))
)
exploded_df = exploded_df.drop("splited").drop("ad_id_details")
exploded_df.show()

+--------------------+--------------------+--------------+----------------+--------------+-----------+-------------+--------------+
|                adid|        package_name|install_market|first_install_dt|last_update_dt|last_use_dt|daily_use_sec|weekly_use_sec|
+--------------------+--------------------+--------------+----------------+--------------+-----------+-------------+--------------+
|0000237c-84d5-42c...|com.google.androi...|             G|      2009010100|    2024082204| 2024083007|            0|          2084|
|0000237c-84d5-42c...|teamdoppelganger....|             G|      2021063018|    2024052017| 2024052017|            0|             0|
|0000237c-84d5-42c...|com.google.androi...|             G|      2009010100|    2024081804| 1970010109|            0|             0|
|0000237c-84d5-42c...|com.samsung.andro...|             S|      2022020601|    2023062301| 1970010109|            0|             0|
|0000237c-84d5-42c...|org.adblockplus.a...|             S|      2021090619| 

In [ ]:
schema_with_comma = StructType(
    [        
        StructField("package_name", StringType(), True),
        StructField("install_market", StringType(), True),
        StructField("first_install_dt", StringType(), True),
        StructField("last_update_dt", StringType(), True),
        StructField("last_use_dt", StringType(), True),
        StructField("daily_use_sec", StringType(), True),
        StructField("weekly_use_sec", StringType(), True),]
)

In [ ]:
schema_with_colon = StructType(
    []
)

In [16]:
        # .option("delimiter", ":")
df = (
    spark.read
        .option("inferSchema", "true")
        .option("multiLine", "true")
        .option("header", "false")
        .option("encoding", "utf-8")
        .csv(data_path, schema=schema)
)

In [20]:
df.printSchema()

root
 |-- ADID: string (nullable = true)
 |-- package_name: string (nullable = true)
 |-- AppID: string (nullable = true)
 |-- installed_market: string (nullable = true)
 |-- initial_installed_dt: string (nullable = true)
 |-- last_update_dt: string (nullable = true)
 |-- daily_usage_time: string (nullable = true)
 |-- weekly_usage_time: string (nullable = true)
 |-- user_age_interval: string (nullable = true)
 |-- user_sex: string (nullable = true)
 |-- Tel_cop: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_os: string (nullable = true)



In [10]:
from IPython.display import display, display_pretty, clear_output, JSON

spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled

In [17]:
display(df)

ADID,package_name,AppID,installed_market,initial_installed_dt,last_update_dt,daily_usage_time,weekly_usage_time,user_age_interval,user_sex,Tel_cop,device_model,device_os
0000237c-84d5-42c...,teamdoppelganger....,com.google.androi...,com.samsung.andro...,org.adblockplus.a...,com.nhn.android.n...,kr.go.hrd.app:G:2...,com.kebhana.hanap...,net.daum.android....,com.seeon.uticket...,com.yagins.hirapo...,net.orizinal.subw...,skplanet.musicmat...
00002a34-4418-430...,kr.co.samsungcard...,com.sec.android.a...,nh.smart.banking:...,com.netflix.media...,com.roadplus.andr...,com.yodo1.crossyr...,com.sec.penup:S:2...,com.cabsoft.oksav...,com.shcard.smartp...,com.sec.android.g...,kr.co.welcomebank...,com.kbcard.cxh.ap...
00018a31-b0f1-4ec...,com.wisetoto:G:20...,com.sec.android.a...,kt.navi:G:2022030...,com.disney.disney...,com.shcard.smartp...,com.google.androi...,com.google.androi...,com.kbcard.cxh.ap...,com.mtelo.ktAPP:E...,com.ktcs.whowho:G...,com.todoist:G:202...,com.kt.olleh.smar...
0001df59-3d34-415...,kr.safemyinfo:G:2...,com.hcom.android:...,com.samsung.andro...,com.fasterpay.app...,com.samsung.andro...,com.skplanet.tmap...,com.mobiletoong.t...,com.sinyee.babybu...,com.brightchamps....,com.google.androi...,com.samsung.andro...,com.sktelecom.tgu...
0002ca60-d565-442...,kr.co.kpcard.palr...,com.samsung.andro...,net.daum.android....,com.croquis.zigza...,com.miraeasset.tr...,kr.or.kotsa.watc:...,com.marchlab.haem...,com.atlassian.and...,com.samsung.andro...,com.byappsc.ggsin...,com.mdd_rn:G:2023...,com.nhnent.payapp...
00032f56-a035-476...,com.sec.android.g...,com.betafish.adbl...,com.google.ar.len...,com.google.androi...,kr.co.lylstudio.h...,com.pinkcore.bb.e...,com.google.androi...,com.google.androi...,com.google.androi...,com.skt.prod.dial...,com.kakao.page:G:...,com.adamrocker.an...
00034f01-be1c-4da...,com.google.androi...,vr.audio.voicerec...,com.kakao.page:G:...,com.hyundai.myhyu...,com.naver.vibe:G:...,kr.go.minwon.m:G:...,com.nhn.android.c...,com.ahnlab.v3mobi...,com.homeplus.myho...,com.cultsotry.yan...,com.microsoft.off...,com.towneers.www:...
0003b234-16b6-40d...,com.sec.android.a...,com.mangosoft.uti...,com.estsoft.alyac...,com.kt.ydatabox:G...,com.instagram.and...,kt.navi:U:2009010...,com.samsung.andro...,com.innov.lotteco...,com.samsung.andro...,com.kt.bellringol...,com.kakao.talk:G:...,com.google.androi...
00040286-9aaf-417...,kr.co.hanamembers...,com.lge.music:E:2...,com.lge.aisuite:E...,lgt.call:O:202001...,kr.co.kbs.kong:G:...,com.skt.tmap.ku:G...,kr.co.nmcs.daejeo...,com.kakao.talk:G:...,kr.go.safepeople:...,com.google.androi...,com.uplus.onphone...,com.nhn.android.s...
0004489b-a47d-455...,com.lcacApp:G:202...,com.facebook.orca...,com.google.androi...,com.samsung.sree:...,com.skt.tmap.ku:O...,com.skmc.okcashba...,com.samsung.andro...,com.epost.insu:G:...,com.bnids.happypa...,com.velox.hkmc_tm...,kr.go.mobileid:G:...,jp.naver.line.and...


In [15]:
display(df.describe())

summary,ADID,package_name,AppID,installed_market,initial_installed_dt,last_update_dt,daily_usage_time,weekly_usage_time,user_age_interval,user_sex,Tel_cop,device_model,device_os
count,114236,114236,114236,114236,114236,114236,114236,114236,114236,114236,114236,114236,114236
mean,NULL,NULL,2.019521642758675E9,2.0231672824757607E9,1.9816831157954147E9,67.62855842291397,NULL,NULL,2.0195457518895795E9,2.023167237609125E9,1.9817240346434662E9,65.31716796806612,NULL
stddev,NULL,NULL,5664172.793053418,2594062.3823616393,2.2237830516064975E7,1093.3100264700295,NULL,NULL,5659980.6914026765,2612477.688017082,2.2266478696805727E7,990.9695613633864,NULL
min,0000237c-84d5-42c...,E,1970010109,1970010109,1969123116,-1,"-1,Com.sktelecom....",E,1970010109,1970010109,1969123116,-1,"-1,Busan.Subway.f..."
max,fffff0ff-9797-45d...,U,2024090104,2024090104,2025061108,997,"999,kr.go.minwon.m",U,2024090102,2024090104,2025071914,999,"999,com.siso.kpcwucn"
